In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error


In [8]:
# Load the train data set
data = pd.read_csv('train.csv')

# Can simplify the Titles with this dictionary:
Title_Dictionary = {"Capt"  : "Officer", 
                    "Col"   : "Officer",
                    "Major" : "Officer",
                    "Dr"    : "Officer", 
                    "Rev"   : "Officer", 
                    "Don"   : "Royalty", 
                    "Sir"   : "Royalty", 
                    "Jonkheer": "Royalty",
                    "the Countess":"Royalty", 
                    "Mr"    : "Mr", 
                    "Mme"   : "Mrs", 
                    "Mrs"   : "Mrs", 
                    "Ms"    : "Miss", 
                    "Miss"  : "Miss", 
                    "Mlle"  : "Miss", 
                    "Master" : "Master", 
                    "Lady"  : "Royalty" 
                    }
    
# adding summarized "Title" column to data
data["Title"] = data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
data["Title"] = data["Title"].map(Title_Dictionary)
# print(data["Title"].value_counts())

for i in ('Sex','Embarked', 'Title'):
    # Create a one-hot encoder
    encoder = OneHotEncoder(handle_unknown='ignore')

    # Encode the "Sex" column
    encoder.fit(data[[i]])
    sex_encoded = encoder.transform(data[[i]])
    sex_encoded_df = pd.DataFrame(sex_encoded.toarray(), columns=encoder.get_feature_names_out([i]))

    # Join the encoded column to the original dataframe
    data = pd.concat([data, sex_encoded_df], axis=1)

    # Drop the original "Sex" column
    data = data.drop(columns=[i])

# # Create a one-hot encoder
# encoder = OneHotEncoder(handle_unknown='ignore')
# # Encode the "Sex" column
# encoder.fit(data[['Sex']])
# sex_encoded = encoder.transform(data[['Sex']])
# # Get the original categories
# original_categories = encoder.categories_[0]
# # Get the new column names
# new_columns = encoder.get_feature_names_out(['Sex'])
# # Map the new column names with the original categories
# new_columns = [f"{new_columns[i]}_{original_categories[i]}" for i in range(len(original_categories))]
# sex_encoded_df = pd.DataFrame(sex_encoded.toarray(), columns=new_columns)
# # Join the encoded column to the original dataframe
# data = pd.concat([data, sex_encoded_df], axis=1)


print(data.columns)
# Drop the original "Sex" column and other non-numerical columns
# train = train.drop(columns=["Name","Sex","Ticket","Cabin","Embarked"])

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Sex_female', 'Sex_male', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Embarked_nan', 'Title_Master',
       'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty'],
      dtype='object')


### with 80/20 train test split

In [9]:
# Perform an 80/20 train test split
data = data.drop(columns=["Name", "Ticket", "Cabin"]).dropna()
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Split the data into y_train and x_train
y_train = train["Survived"]
x_train = train.drop(columns=["Survived"])

y_test = test["Survived"]
x_test = test.drop(columns=["Survived"])

# print(y_train.columns)
# print(y_test.columns)
print(x_train.columns)
print(x_test.columns)
# print(train.columns)
# print(test.columns)

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_nan',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_nan',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty'],
      dtype='object')


In [10]:
# Create a Random Forest model
RF = RandomForestClassifier(random_state=1)

# Use RFE (Recursive Factor Elimination) to select the top 4 features
my_model = RFE(estimator=RF, n_features_to_select=5)
my_model.fit(x_train, y_train)

# Print the selected features
print(x_train.columns[my_model.support_])

# Fit the model to the x_train and y_train data
my_model.fit(x_train, y_train)

# Make predictions for the test data
y_test_pred = my_model.predict(x_test)

# Measure the accuracy of the model
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Model accuracy: {accuracy}")


Index(['PassengerId', 'Age', 'Fare', 'Sex_female', 'Title_Mr'], dtype='object')
Model accuracy: 0.7762237762237763


In [11]:
# Create an XGBoost model
xgb = XGBClassifier(random_state=1)

# Use RFE (Recursive Factor Elimination) to select the top 4 features
my_model = RFE(estimator=xgb, n_features_to_select=5)
my_model.fit(x_train, y_train)

# Print the selected features
print(x_train.columns[my_model.support_])

# Fit the model to the x_train and y_train data
my_model.fit(x_train, y_train)

# Make predictions for the test data
y_test_pred = my_model.predict(x_test)

# Measure the accuracy of the model
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Model accuracy: {accuracy}")

Index(['Pclass', 'SibSp', 'Sex_female', 'Title_Master', 'Title_Mr'], dtype='object')
Model accuracy: 0.8041958041958042


In [12]:
df = pd.DataFrame(y_test_pred, columns = ['Survived'])
df.info()

submission = pd.DataFrame({"PassengerId" : test['PassengerId'], "Survived" : y_test_pred}).astype('int32')
submission.info()

submission.to_csv("submission_XGB.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Survived  143 non-null    int64
dtypes: int64(1)
memory usage: 1.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 149 to 136
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  143 non-null    int32
 1   Survived     143 non-null    int32
dtypes: int32(2)
memory usage: 2.2 KB


In [21]:
# NOTE: could use this to calculate MSE & RMSE  if I had the sale price for the test data too.
import math
import numpy as np

mse = mean_squared_error(np.array(y_test), np.array(y_test_pred["Survived"]))
# rmse = sqrt(mse)
print(mse)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices